# Extract Atmospec Centroid information in OGA

 work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/11/02
- update : 2022/11/02


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [2]:
! eups list -s lsst_distrib

   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [3]:
import lsst.daf.butler as dafButler
import lsst.summit.utils.butlerUtils as butlerUtils
import lsst.daf.base as dafBase

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Pipeline tasks from DP0.2
#from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
#from lsst.meas.algorithms.detection import SourceDetectionTask
#from lsst.meas.deblender import SourceDeblendTask
#from lsst.meas.base import SingleFrameMeasurementTask

# fromm StackClub on Britter-Fatter
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
import lsst.meas.extensions.shapeHSM
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig

In [6]:
#Set the matplotlib plot color table.
plt.style.use('tableau-colorblind10')

In [7]:
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (6.0, 6.0)

In [8]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## Give the collection

In [9]:
my_collection = "u/dagoret/spectro/noflat/empty~holo4/20220912"

In [10]:
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Config

In [11]:
# path index for each month
DATE="20220912"
filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

## Select the atmospecCentroid

In [12]:
datasetRefs = registry.queryDatasets(datasetType='atmospecCentroid', collections=my_collection, where= "instrument='LATISS'")

In [13]:
all_dataId = []
all_atmcentroid  = []
all_exposures = []



for i, ref in enumerate(datasetRefs):
   
    print(f"========({i})================datasetType = atmospecCentroid ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["visit"]
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    #spec       = butler.get('spectraction',the_dataId)
    atmcentroid = butler.get('atmospecCentroid', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_atmcentroid.append(atmcentroid)
    #if i >= 0:
    #    break

========(0)================datasetType = atmospecCentroid ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2022091200106}
visit................: 2022091200106
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/spectro/noflat/empty~holo4/20220912/20221026T100113Z
{'day_obs': 20220912, 'seq_num': 106, 'detector': 0}
========(1)================datasetType = atmospecCentroid ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2022091200107}
visit................: 2022091200107
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/spectro/noflat/empty~holo4/20220912/20221026T101538Z
{'day_obs': 20220912, 'seq_num': 107, 'detector': 0}
========(2)=======

In [14]:
all_exposures

[2022091200106,
 2022091200107,
 2022091200113,
 2022091200114,
 2022091200121,
 2022091200122,
 2022091200128,
 2022091200129,
 2022091200135,
 2022091200136,
 2022091200142,
 2022091200143,
 2022091200149,
 2022091200150,
 2022091200158,
 2022091200159,
 2022091200165,
 2022091200166,
 2022091200174,
 2022091200175,
 2022091200181,
 2022091200182,
 2022091200188,
 2022091200665,
 2022091200189,
 2022091200196,
 2022091200197,
 2022091200203,
 2022091200204,
 2022091200210,
 2022091200211,
 2022091200217,
 2022091200218,
 2022091200224,
 2022091200672,
 2022091200225,
 2022091200231,
 2022091200232,
 2022091200238,
 2022091200239,
 2022091200245,
 2022091200246,
 2022091200252,
 2022091200253,
 2022091200259,
 2022091200260,
 2022091200266,
 2022091200267,
 2022091200273,
 2022091200274,
 2022091200280,
 2022091200281,
 2022091200671,
 2022091200287,
 2022091200288,
 2022091200294,
 2022091200295,
 2022091200304,
 2022091200304,
 2022091200346,
 2022091200304,
 2022091200354,
 2022091

In [15]:
all_atmcentroid 

[{'astrometricMatch': False, 'centroid': [1678.0, 301.0]},
 {'astrometricMatch': False,
  'centroid': [1680.8971922803162, 310.1231372035838]},
 {'astrometricMatch': False, 'centroid': [1660.0, 303.0]},
 {'astrometricMatch': False, 'centroid': [1661.0, 298.0]},
 {'astrometricMatch': False, 'centroid': [1689.0, 304.0]},
 {'astrometricMatch': False, 'centroid': [1685.0, 310.0]},
 {'astrometricMatch': False,
  'centroid': [1679.6461830799772, 308.3663370359814]},
 {'astrometricMatch': False, 'centroid': [1674.0, 302.0]},
 {'astrometricMatch': False, 'centroid': [1657.0, 298.0]},
 {'astrometricMatch': False, 'centroid': [1680.0, 300.0]},
 {'astrometricMatch': False, 'centroid': [1675.0, 301.0]},
 {'astrometricMatch': True,
  'centroid': [1681.4455716085636, 308.19066449929755]},
 {'astrometricMatch': False, 'centroid': [1665.0, 296.0]},
 {'astrometricMatch': False, 'centroid': [1666.0, 297.0]},
 {'astrometricMatch': False, 'centroid': [1682.0, 299.0]},
 {'astrometricMatch': False, 'centroi